# Benchmarking Different dataframe libraries

- https://www.youtube.com/watch?v=zrY2u2_WJ0o
- Packages to compare:
    - Pandas 2.0 (pyarrow)
    - Pandas + cudf
    - Polars (python api)

1. Gather dataset (large GB+)
    - Different data types (date, int, floats, strings, etc.)
    - https://data.cms.gov/provider-summary-by-type-of-service/medicare-physician-other-practitioners/medicare-physician-other-practitioners-by-provider/data
2. install latest of each library
3. create benchmarks to run
4. test

In [1]:
%pip install polars

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import polars as pl
import requests
import sys
import os

print(pd.__version__)
print(pl.__version__)

2.1.4
0.20.2


# Data

## Download data

In [3]:
# if medicare_data.csv doesn't exist, download it 
if not os.path.exists('medicare_data.csv'):
    url = 'https://data.cms.gov/sites/default/files/2023-05/914a4463-7af3-423f-83a7-b343794e20ee/MUP_PHY_R23_P05_V10_D21_Prov_Svc.csv'
    response = requests.get(url)
    if response.status_code == 200:
        with open('medicare_data.csv', 'wb') as file:
            file.write(response.content)
        print("File downloaded successfully.")
    else:
        print("Failed to download the file.")

## pandas df (default backend)

In [4]:
pandas_df = pd.read_csv('medicare_data.csv')
pandas_df.shape

/tmp/ipykernel_5987/330544136.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  pandas_df = pd.read_csv('medicare_data.csv')


(9886177, 29)

In [5]:
print(f"{sys.getsizeof(pandas_df) / (1024**3): .2f} GB")

 12.65 GB


In [6]:
pandas_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9886177 entries, 0 to 9886176
Data columns (total 29 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   Rndrng_NPI                     int64  
 1   Rndrng_Prvdr_Last_Org_Name     object 
 2   Rndrng_Prvdr_First_Name        object 
 3   Rndrng_Prvdr_MI                object 
 4   Rndrng_Prvdr_Crdntls           object 
 5   Rndrng_Prvdr_Gndr              object 
 6   Rndrng_Prvdr_Ent_Cd            object 
 7   Rndrng_Prvdr_St1               object 
 8   Rndrng_Prvdr_St2               object 
 9   Rndrng_Prvdr_City              object 
 10  Rndrng_Prvdr_State_Abrvtn      object 
 11  Rndrng_Prvdr_State_FIPS        object 
 12  Rndrng_Prvdr_Zip5              object 
 13  Rndrng_Prvdr_RUCA              float64
 14  Rndrng_Prvdr_RUCA_Desc         object 
 15  Rndrng_Prvdr_Cntry             object 
 16  Rndrng_Prvdr_Type              object 
 17  Rndrng_Prvdr_Mdcr_Prtcptg_Ind  object 
 18  HC

In [7]:
pandas_df['Rndrng_Prvdr_Gndr'] = pandas_df['Rndrng_Prvdr_Gndr'].astype('category')
pandas_df['Rndrng_Prvdr_State_FIPS'] = pandas_df['Rndrng_Prvdr_State_FIPS'].\
    astype('str').\
    astype('category')
pandas_df['Rndrng_Prvdr_Mdcr_Prtcptg_Ind'] = pandas_df['Rndrng_Prvdr_Mdcr_Prtcptg_Ind'].\
    replace({'Y': 1, 'N': 0}).\
    astype('boolean')
pandas_df['Rndrng_Prvdr_Zip5'] = pandas_df['Rndrng_Prvdr_Zip5'].astype('str')

In [8]:
pandas_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9886177 entries, 0 to 9886176
Data columns (total 29 columns):
 #   Column                         Dtype   
---  ------                         -----   
 0   Rndrng_NPI                     int64   
 1   Rndrng_Prvdr_Last_Org_Name     object  
 2   Rndrng_Prvdr_First_Name        object  
 3   Rndrng_Prvdr_MI                object  
 4   Rndrng_Prvdr_Crdntls           object  
 5   Rndrng_Prvdr_Gndr              category
 6   Rndrng_Prvdr_Ent_Cd            object  
 7   Rndrng_Prvdr_St1               object  
 8   Rndrng_Prvdr_St2               object  
 9   Rndrng_Prvdr_City              object  
 10  Rndrng_Prvdr_State_Abrvtn      object  
 11  Rndrng_Prvdr_State_FIPS        category
 12  Rndrng_Prvdr_Zip5              object  
 13  Rndrng_Prvdr_RUCA              float64 
 14  Rndrng_Prvdr_RUCA_Desc         object  
 15  Rndrng_Prvdr_Cntry             object  
 16  Rndrng_Prvdr_Type              object  
 17  Rndrng_Prvdr_Mdcr_Prtcptg_I

## Build pandas dataframe with Pyarrow backend

In [9]:
pandas_df.to_parquet('medicare_data.parquet')

In [ ]:
import os
print(f"{os.path.getsize('medicare_data.csv') / (1024**3): .2f} GB")
print(f"{os.path.getsize('medicare_data.parquet') / (1024**3): .2f} GB")

In [ ]:
# read in data with pyarrow backend
pyarrow_df = pd.read_parquet('medicare_data.parquet', dtype_backend='pyarrow')

In [ ]:
# 11.4 GB vs 3.5 GB
pyarrow_df.info(memory_usage='deep')

# Dask

In [ ]:
import dask.dataframe as dd

dask_df = dd.from_pandas(pandas_df, npartitions=10)
dask_df.info()

## Polars

In [ ]:
%pip install polars

In [ ]:
# polars_df = pl.from_pandas(pandas_df)
polars_df_scan = pl.scan_parquet('medicare_data.parquet')
polars_df_read = pl.read_parquet('medicare_data.parquet')

## pandas cuDF

In [ ]:
# %pip install cudf-cu12 rmm-cu12 dask-cudf-cu12 --extra-index-url=https://pypi.ngc.nvidia.com/
# %pip install --no-cache-dir --extra-index-url https://pypi.nvidia.com cudf-cu12
# %pip install \
#     --extra-index-url=https://pypi.nvidia.com \
#     cudf-cu12==23.12.* dask-cudf-cu12==23.12.* cuml-cu12==23.12.* \
#     cugraph-cu12==23.12.* cuspatial-cu12==23.12.* cuproj-cu12==23.12.* \
#     cuxfilter-cu12==23.12.* cucim-cu12==23.12.* pylibraft-cu12==23.12.* \
#     raft-dask-cu12==23.12.*

---

# Benchmarks

In [ ]:
# pyarrow slower than pandas
%timeit pandas_df['Rndrng_Prvdr_State_FIPS'].value_counts()
%timeit pyarrow_df['Rndrng_Prvdr_State_FIPS'].value_counts()
%timeit polars_df_read['Rndrng_Prvdr_State_FIPS'].value_counts().sort(-pl.col('count'))
%timeit polars_df_scan.group_by('Rndrng_Prvdr_State_FIPS').agg(pl.count()).sort(-pl.col('count')).collect()

In [ ]:
# why dask so slow?
%timeit dask_df['Rndrng_Prvdr_State_FIPS'].value_counts().compute()

---

In [ ]:
# pyarrow faster than pandas
%timeit pandas_df.groupby('Rndrng_Prvdr_Crdntls')['Avg_Sbmtd_Chrg'].mean().sort_values()
%timeit pyarrow_df.groupby('Rndrng_Prvdr_Crdntls')['Avg_Sbmtd_Chrg'].mean().sort_values()

In [ ]:
%timeit pandas_df.query("Rndrng_Prvdr_Crdntls == '(FNP) FAMILY NURSE P'")
%timeit pyarrow_df.query("Rndrng_Prvdr_Crdntls == '(FNP) FAMILY NURSE P'")

In [ ]:
# dask is terrible for querying
%timeit dask_df.query("Rndrng_Prvdr_Crdntls == '(FNP) FAMILY NURSE P'").compute()

---

In [ ]:
# observed=False to remove warnings
%timeit pandas_df.\
    groupby(['Rndrng_Prvdr_Crdntls', 'Rndrng_Prvdr_Gndr'], observed=False)\
    [['Tot_Benes', 'Tot_Srvcs', 'Tot_Bene_Day_Srvcs', 'Avg_Sbmtd_Chrg', 'Avg_Mdcr_Alowd_Amt', 'Avg_Mdcr_Pymt_Amt', 'Avg_Mdcr_Stdzd_Amt']].\
    agg(['mean', 'std', 'min', 'max'])
%timeit pyarrow_df.\
    groupby(['Rndrng_Prvdr_Crdntls', 'Rndrng_Prvdr_Gndr'], observed=False)\
    [['Tot_Benes', 'Tot_Srvcs', 'Tot_Bene_Day_Srvcs', 'Avg_Sbmtd_Chrg', 'Avg_Mdcr_Alowd_Amt', 'Avg_Mdcr_Pymt_Amt', 'Avg_Mdcr_Stdzd_Amt']].\
    agg(['mean', 'std', 'min', 'max'])

In [ ]:
%%timeit

results = polars_df_read.group_by(['Rndrng_Prvdr_Crdntls', 'Rndrng_Prvdr_Gndr']).\
    agg([
        pl.col('Tot_Benes').mean().alias('Tot_Benes_mean'),
        pl.col('Tot_Benes').std().alias('Tot_Benes_std'),
        pl.col('Tot_Benes').min().alias('Tot_Benes_min'),
        pl.col('Tot_Benes').max().alias('Tot_Benes_max'),

        pl.col('Tot_Srvcs').mean().alias('Tot_Srvcs_mean'),
        pl.col('Tot_Srvcs').std().alias('Tot_Srvcs_std'),
        pl.col('Tot_Srvcs').min().alias('Tot_Srvcs_min'),
        pl.col('Tot_Srvcs').max().alias('Tot_Srvcs_max'),

        pl.col('Tot_Bene_Day_Srvcs').mean().alias('Tot_Bene_Day_Srvcs_mean'),
        pl.col('Tot_Bene_Day_Srvcs').std().alias('Tot_Bene_Day_Srvcs_std'),
        pl.col('Tot_Bene_Day_Srvcs').min().alias('Tot_Bene_Day_Srvcs_min'),
        pl.col('Tot_Bene_Day_Srvcs').max().alias('Tot_Bene_Day_Srvcs_max'),

        pl.col('Avg_Sbmtd_Chrg').mean().alias('Avg_Sbmtd_Chrg_mean'),
        pl.col('Avg_Sbmtd_Chrg').std().alias('Avg_Sbmtd_Chrg_std'),
        pl.col('Avg_Sbmtd_Chrg').min().alias('Avg_Sbmtd_Chrg_min'),
        pl.col('Avg_Sbmtd_Chrg').max().alias('Avg_Sbmtd_Chrg_max'),

        pl.col('Avg_Mdcr_Alowd_Amt').mean().alias('Avg_Mdcr_Alowd_Amt_mean'),
        pl.col('Avg_Mdcr_Alowd_Amt').std().alias('Avg_Mdcr_Alowd_Amt_std'),
        pl.col('Avg_Mdcr_Alowd_Amt').min().alias('Avg_Mdcr_Alowd_Amt_min'),
        pl.col('Avg_Mdcr_Alowd_Amt').max().alias('Avg_Mdcr_Alowd_Amt_max'),

        pl.col('Avg_Mdcr_Pymt_Amt').mean().alias('Avg_Mdcr_Pymt_Amt_mean'),
        pl.col('Avg_Mdcr_Pymt_Amt').std().alias('Avg_Mdcr_Pymt_Amt_std'),
        pl.col('Avg_Mdcr_Pymt_Amt').min().alias('Avg_Mdcr_Pymt_Amt_min'),
        pl.col('Avg_Mdcr_Pymt_Amt').max().alias('Avg_Mdcr_Pymt_Amt_max'),

        pl.col('Avg_Mdcr_Stdzd_Amt').mean().alias('Avg_Mdcr_Stdzd_Amt_mean'),
        pl.col('Avg_Mdcr_Stdzd_Amt').std().alias('Avg_Mdcr_Stdzd_Amt_std'),
        pl.col('Avg_Mdcr_Stdzd_Amt').min().alias('Avg_Mdcr_Stdzd_Amt_min'),
        pl.col('Avg_Mdcr_Stdzd_Amt').max().alias('Avg_Mdcr_Stdzd_Amt_max'),
    ]).collect()

In [ ]:
%%timeit

results = polars_df_scan.group_by(['Rndrng_Prvdr_Crdntls', 'Rndrng_Prvdr_Gndr']).\
    agg([
        pl.col('Tot_Benes').mean().alias('Tot_Benes_mean'),
        pl.col('Tot_Benes').std().alias('Tot_Benes_std'),
        pl.col('Tot_Benes').min().alias('Tot_Benes_min'),
        pl.col('Tot_Benes').max().alias('Tot_Benes_max'),

        pl.col('Tot_Srvcs').mean().alias('Tot_Srvcs_mean'),
        pl.col('Tot_Srvcs').std().alias('Tot_Srvcs_std'),
        pl.col('Tot_Srvcs').min().alias('Tot_Srvcs_min'),
        pl.col('Tot_Srvcs').max().alias('Tot_Srvcs_max'),

        pl.col('Tot_Bene_Day_Srvcs').mean().alias('Tot_Bene_Day_Srvcs_mean'),
        pl.col('Tot_Bene_Day_Srvcs').std().alias('Tot_Bene_Day_Srvcs_std'),
        pl.col('Tot_Bene_Day_Srvcs').min().alias('Tot_Bene_Day_Srvcs_min'),
        pl.col('Tot_Bene_Day_Srvcs').max().alias('Tot_Bene_Day_Srvcs_max'),

        pl.col('Avg_Sbmtd_Chrg').mean().alias('Avg_Sbmtd_Chrg_mean'),
        pl.col('Avg_Sbmtd_Chrg').std().alias('Avg_Sbmtd_Chrg_std'),
        pl.col('Avg_Sbmtd_Chrg').min().alias('Avg_Sbmtd_Chrg_min'),
        pl.col('Avg_Sbmtd_Chrg').max().alias('Avg_Sbmtd_Chrg_max'),

        pl.col('Avg_Mdcr_Alowd_Amt').mean().alias('Avg_Mdcr_Alowd_Amt_mean'),
        pl.col('Avg_Mdcr_Alowd_Amt').std().alias('Avg_Mdcr_Alowd_Amt_std'),
        pl.col('Avg_Mdcr_Alowd_Amt').min().alias('Avg_Mdcr_Alowd_Amt_min'),
        pl.col('Avg_Mdcr_Alowd_Amt').max().alias('Avg_Mdcr_Alowd_Amt_max'),

        pl.col('Avg_Mdcr_Pymt_Amt').mean().alias('Avg_Mdcr_Pymt_Amt_mean'),
        pl.col('Avg_Mdcr_Pymt_Amt').std().alias('Avg_Mdcr_Pymt_Amt_std'),
        pl.col('Avg_Mdcr_Pymt_Amt').min().alias('Avg_Mdcr_Pymt_Amt_min'),
        pl.col('Avg_Mdcr_Pymt_Amt').max().alias('Avg_Mdcr_Pymt_Amt_max'),

        pl.col('Avg_Mdcr_Stdzd_Amt').mean().alias('Avg_Mdcr_Stdzd_Amt_mean'),
        pl.col('Avg_Mdcr_Stdzd_Amt').std().alias('Avg_Mdcr_Stdzd_Amt_std'),
        pl.col('Avg_Mdcr_Stdzd_Amt').min().alias('Avg_Mdcr_Stdzd_Amt_min'),
        pl.col('Avg_Mdcr_Stdzd_Amt').max().alias('Avg_Mdcr_Stdzd_Amt_max'),
    ]).collect()

In [ ]:
polars_df_scan.group_by(['Rndrng_Prvdr_Crdntls', 'Rndrng_Prvdr_Gndr']).\
    agg([
        pl.col('Tot_Benes').mean().alias('Tot_Benes_mean'),
        pl.col('Tot_Benes').std().alias('Tot_Benes_std'),
        pl.col('Tot_Benes').min().alias('Tot_Benes_min'),
        pl.col('Tot_Benes').max().alias('Tot_Benes_max'),

        pl.col('Tot_Srvcs').mean().alias('Tot_Srvcs_mean'),
        pl.col('Tot_Srvcs').std().alias('Tot_Srvcs_std'),
        pl.col('Tot_Srvcs').min().alias('Tot_Srvcs_min'),
        pl.col('Tot_Srvcs').max().alias('Tot_Srvcs_max'),

        pl.col('Tot_Bene_Day_Srvcs').mean().alias('Tot_Bene_Day_Srvcs_mean'),
        pl.col('Tot_Bene_Day_Srvcs').std().alias('Tot_Bene_Day_Srvcs_std'),
        pl.col('Tot_Bene_Day_Srvcs').min().alias('Tot_Bene_Day_Srvcs_min'),
        pl.col('Tot_Bene_Day_Srvcs').max().alias('Tot_Bene_Day_Srvcs_max'),

        pl.col('Avg_Sbmtd_Chrg').mean().alias('Avg_Sbmtd_Chrg_mean'),
        pl.col('Avg_Sbmtd_Chrg').std().alias('Avg_Sbmtd_Chrg_std'),
        pl.col('Avg_Sbmtd_Chrg').min().alias('Avg_Sbmtd_Chrg_min'),
        pl.col('Avg_Sbmtd_Chrg').max().alias('Avg_Sbmtd_Chrg_max'),

        pl.col('Avg_Mdcr_Alowd_Amt').mean().alias('Avg_Mdcr_Alowd_Amt_mean'),
        pl.col('Avg_Mdcr_Alowd_Amt').std().alias('Avg_Mdcr_Alowd_Amt_std'),
        pl.col('Avg_Mdcr_Alowd_Amt').min().alias('Avg_Mdcr_Alowd_Amt_min'),
        pl.col('Avg_Mdcr_Alowd_Amt').max().alias('Avg_Mdcr_Alowd_Amt_max'),

        pl.col('Avg_Mdcr_Pymt_Amt').mean().alias('Avg_Mdcr_Pymt_Amt_mean'),
        pl.col('Avg_Mdcr_Pymt_Amt').std().alias('Avg_Mdcr_Pymt_Amt_std'),
        pl.col('Avg_Mdcr_Pymt_Amt').min().alias('Avg_Mdcr_Pymt_Amt_min'),
        pl.col('Avg_Mdcr_Pymt_Amt').max().alias('Avg_Mdcr_Pymt_Amt_max'),

        pl.col('Avg_Mdcr_Stdzd_Amt').mean().alias('Avg_Mdcr_Stdzd_Amt_mean'),
        pl.col('Avg_Mdcr_Stdzd_Amt').std().alias('Avg_Mdcr_Stdzd_Amt_std'),
        pl.col('Avg_Mdcr_Stdzd_Amt').min().alias('Avg_Mdcr_Stdzd_Amt_min'),
        pl.col('Avg_Mdcr_Stdzd_Amt').max().alias('Avg_Mdcr_Stdzd_Amt_max'),
    ]).collect()

In [ ]:
pandas_df.\
    groupby(['Rndrng_Prvdr_Crdntls', 'Rndrng_Prvdr_Gndr'], observed=False)\
    [['Tot_Benes', 'Tot_Srvcs', 'Tot_Bene_Day_Srvcs', 'Avg_Sbmtd_Chrg', 'Avg_Mdcr_Alowd_Amt', 'Avg_Mdcr_Pymt_Amt', 'Avg_Mdcr_Stdzd_Amt']].\
    agg(['mean', 'std', 'min', 'max'])

---

# Appendix

In [ ]:
# url = 'https://data.cms.gov/data-api/v1/dataset/5a6f0f6f-0439-403d-bd99-2c7631003cb1/data-viewer?_format=csv'

# response = requests.get(url)
# # Check if the request was successful
# if response.status_code == 200:
#     # Use io.BytesIO to treat the response content as a file-like object for zipfile
#     zipped_file = zipfile.ZipFile(io.BytesIO(response.content))
#     # Extract all the contents into the current directory
#     zipped_file.extractall()
#     print("File downloaded and unzipped successfully.")
# else:
#     print("Failed to download the file.")
# pandas_df = pd.read_csv('Medicare_Physician_Other_Practitioners_by_Provider_2021.csv')
# pandas_df.shape